<a href="https://colab.research.google.com/github/anshulsawant/WhatDoesThisReallyDo/blob/main/What_does_this_function_really_do_JAX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## My attempt to address my biggest numpy fear: I assume that this function is processing stuff in some way, but what if it isn't. I hope this is not going to come crashing down.
import jax
from jax import numpy as np

In [13]:
## jax.lax.select
x = np.reshape(np.arange(6)*2 + 1, (2,3))
y = np.reshape(np.arange(6)*2, (2,3))
p = np.reshape(np.array([True, False, True, False, True, False]), (2,3))
z = jax.lax.select(p, x, y)
assert np.all(z == x*p + y*(1-p))
x,y,z

(Array([[ 1,  3,  5],
        [ 7,  9, 11]], dtype=int32),
 Array([[ 0,  2,  4],
        [ 6,  8, 10]], dtype=int32),
 Array([[ 1,  2,  5],
        [ 6,  9, 10]], dtype=int32))